<a href="https://colab.research.google.com/github/satish0110/Encryptix/blob/main/Credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'creditcardfraud:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F310%2F23498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240831%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240831T052338Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daf809113f827ba54ad5fc916c7ae9e142ef4d44789cd2d91380c76fe114390a367d809c5e4583646a942ecb48a4b4792c4cf5325acdd76cabc3fc9e5b8114fa3b18916136ab59c7730b987ed03e6f2a225945e79cb244a7f891c0c628e112cfcb7a53bf7ed33c6dc4b8305126b2bbadd1232605696e50736941b07f18df47048f7d054ac9918466713bdf3ff5985e0d9d0fe28e181762edb4632821bd93509809b628422c431041a8b0e3584d5196ee60e62a99ae9dfc65a193390e246b1a196703dcc6171ae277ab9f4b3795e2b21057f435c785a228c5b342513ddeaa70843e3a19a081127a674f1bab4d5b687d5730f75a1d9c2f8818704e67a1170a7099d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:

import numpy as ny
import pandas as ps
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [ ]:
dt =ps.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [ ]:
dt.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
dt.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

1= Fraud ,
0= normal transaction

In [ ]:
dt["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

**preprocessing data**

In [ ]:
#separate the data
n=dt[dt["Class"]==0]
f=dt[dt["Class"]==1]

In [ ]:
n.shape

(284315, 31)

In [ ]:
f.shape

(492, 31)

In [ ]:
n["Amount"].describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [ ]:
f["Amount"].describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [ ]:
dt_class=dt.groupby("Class").mean()

In [ ]:
dt_class

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Under sampling the dataset

In [ ]:
n=n.sample(n=492)

In [ ]:
new_dt = ps.concat([n, f], axis=0)

In [ ]:
new_dt

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
277338,167599.0,-1.198057,1.068628,0.432871,0.578752,0.996599,-0.837583,1.255982,-0.262513,-0.146392,...,0.011126,0.609820,-0.096325,-0.019346,0.673976,-0.443096,0.250681,0.131341,28.56,0
235131,148262.0,-3.406086,-0.277414,-0.006971,-3.034248,-0.947527,-0.809767,-0.767003,1.202750,-0.765520,...,-0.391759,-0.577357,0.237621,-0.392304,0.750944,0.553298,-0.466060,-0.271156,22.00,0
198504,132516.0,-1.270778,-0.381040,1.091023,-2.110014,-0.477729,-0.695855,-0.618236,0.522338,-1.038519,...,0.529561,1.153600,-0.274832,0.049728,-0.083172,-0.217684,-0.147801,0.045298,19.95,0
173245,121401.0,1.919182,-1.742647,0.775662,-0.260260,-2.179314,0.421375,-1.934811,0.253900,1.537901,...,0.333796,1.337362,0.046490,0.077855,-0.235563,0.014173,0.086364,-0.020096,79.00,0
222911,143174.0,2.189281,-0.727144,-1.453394,-1.071583,-0.333891,-0.876030,-0.355614,-0.302825,-0.863469,...,0.472976,1.361456,-0.091372,-0.326797,0.243627,0.069989,-0.041225,-0.076403,25.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [ ]:
new_dt.Class.value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [ ]:
new_dt_g=new_dt.groupby("Class").mean()

In [ ]:
new_dt_g

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,99338.101626,-0.096918,0.024178,-0.105379,-0.006059,0.088591,-0.023598,0.072718,-0.026442,-0.051611,...,0.023770,0.052560,-0.001036,0.042845,-0.039633,-0.024804,0.027070,-0.045466,-0.000373,91.190285
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [ ]:
new_dt

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
277338,167599.0,-1.198057,1.068628,0.432871,0.578752,0.996599,-0.837583,1.255982,-0.262513,-0.146392,...,0.011126,0.609820,-0.096325,-0.019346,0.673976,-0.443096,0.250681,0.131341,28.56,0
235131,148262.0,-3.406086,-0.277414,-0.006971,-3.034248,-0.947527,-0.809767,-0.767003,1.202750,-0.765520,...,-0.391759,-0.577357,0.237621,-0.392304,0.750944,0.553298,-0.466060,-0.271156,22.00,0
198504,132516.0,-1.270778,-0.381040,1.091023,-2.110014,-0.477729,-0.695855,-0.618236,0.522338,-1.038519,...,0.529561,1.153600,-0.274832,0.049728,-0.083172,-0.217684,-0.147801,0.045298,19.95,0
173245,121401.0,1.919182,-1.742647,0.775662,-0.260260,-2.179314,0.421375,-1.934811,0.253900,1.537901,...,0.333796,1.337362,0.046490,0.077855,-0.235563,0.014173,0.086364,-0.020096,79.00,0
222911,143174.0,2.189281,-0.727144,-1.453394,-1.071583,-0.333891,-0.876030,-0.355614,-0.302825,-0.863469,...,0.472976,1.361456,-0.091372,-0.326797,0.243627,0.069989,-0.041225,-0.076403,25.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


Spliting the data into features and target

In [ ]:
x=new_dt.drop(columns='Class',axis=1)
y=new_dt["Class"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [ ]:
print(x.shape, x_train.shape, x_test.shape)

(984, 30) (787, 30) (197, 30)


In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train, y_train)

LogisticRegression()

In [ ]:
# accuracy on training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [ ]:
print(f'Accuracy on Training data : , {training_data_accuracy}')


Accuracy on Training data : , 0.9301143583227446


In [ ]:
# accuracy on test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [ ]:
print(f'Accuracy score on Test Data : , {test_data_accuracy}')

Accuracy score on Test Data : , 0.9238578680203046


In [ ]:
min_values = new_dt.describe().loc['min']
max_values = new_dt.describe().loc['max']

print("Minimum Values:\n", min_values)
print("\nMaximum Values:\n", max_values)

Minimum Values:
 Time      315.000000
V1        -30.552380
V2        -17.637853
V3        -31.103685
V4         -4.054508
V5        -22.105532
V6         -6.406267
V7        -43.557242
V8        -41.044261
V9        -13.434066
V10       -24.588262
V11        -2.831366
V12       -18.683715
V13        -3.127795
V14       -19.214325
V15        -4.498945
V16       -14.129855
V17       -25.162799
V18        -9.498746
V19        -3.681904
V20        -7.605660
V21       -22.797604
V22        -8.887017
V23       -19.254328
V24        -2.727935
V25        -4.781606
V26        -1.155110
V27        -7.263482
V28        -2.287800
Amount      0.000000
Class       0.000000
Name: min, dtype: float64

Maximum Values:
 Time      172459.000000
V1             2.330924
V2            22.057729
V3             3.222936
V4            12.114672
V5            11.095089
V6             6.474115
V7            10.949696
V8            20.007208
V9             5.278721
V10            6.609618
V11           12.018913


In [ ]:
# Define the minimum and maximum values for each feature
min_values = {
    'Time': 266.000000, 'V1': -30.552380, 'V2': -14.505648, 'V3': -31.103685, 'V4': -4.399240,
    'V5': -22.105532, 'V6': -10.136371, 'V7': -43.557242, 'V8': -41.044261, 'V9': -13.434066,
    'V10': -24.588262, 'V11': -2.759528, 'V12': -18.683715, 'V13': -3.127795, 'V14': -19.214325,
    'V15': -4.498945, 'V16': -14.129855, 'V17': -25.162799, 'V18': -9.498746, 'V19': -3.681904,
    'V20': -7.642808, 'V21': -22.797604, 'V22': -8.887017, 'V23': -19.254328, 'V24': -2.100164,
    'V25': -4.781606, 'V26': -1.443139, 'V27': -7.263482, 'V28': -8.307955, 'Amount': 0.000000
}

max_values = {
    'Time': 172575.000000, 'V1': 2.353031, 'V2': 22.057729, 'V3': 3.358655, 'V4': 12.114672,
    'V5': 13.367253, 'V6': 6.474115, 'V7': 5.802537, 'V8': 20.007208, 'V9': 4.619855,
    'V10': 6.982557, 'V11': 12.018913, 'V12': 2.660866, 'V13': 3.494972, 'V14': 4.047301,
    'V15': 2.471358, 'V16': 3.139656, 'V17': 6.739384, 'V18': 3.790316, 'V19': 5.228342,
    'V20': 11.059004, 'V21': 27.202839, 'V22': 8.361985, 'V23': 5.466230, 'V24': 1.237015,
    'V25': 2.208209, 'V26': 2.745261, 'V27': 4.610936, 'V28': 4.106596, 'Amount': 2290.050000
}


In [ ]:
# Function to generate a random number between min and max values
def generate_random_data(min_vals, max_vals):
    data = {feature: ny.random.uniform(min_vals[feature], max_vals[feature]) for feature in min_vals}
    return ps.DataFrame([data])

In [ ]:
new_data_dt = generate_random_data(min_values, max_values)

In [ ]:
prediction = model.predict(new_data_dt)

# Print the prediction
if prediction[0] == 1:
    print("Fraud.")
else:
    print("Not fraud.")

The transaction is fraudulent.
